In [1]:
## please delete below code after convertion in converted script(py) file
## + 필요없는 내용 삭제(초반부 1,3~14열, In[ ]형태의 주석제거)
!jupyter nbconvert --to script augment_cells.ipynb
!sed -i '/^#[ ]In\[/d' augment_cells.py
!sed -i -e '1d;3,14d' augment_cells.py

[NbConvertApp] Converting notebook augment_cells.ipynb to script
[NbConvertApp] Writing 1566 bytes to augment_cells.py


In [ ]:
import torch
import torch.nn as nn
from models import ops
import genotypes as gt

In [ ]:
class AugmentCell(nn.Module):
    def __init__(self, genotype, C_pp, C_p, C, reduction_p, reduction):
        super().__init__()
        self.reduction = reduction
        self.n_nodes = len(genotype.normal)
        
        if reduction_p:
            self.preproc0 = ops.FactorizedReduce(C_pp, C)
        else:
            self.preproc0 = ops.StdConv(C_pp, C, 1, 1, 0)
        self.preproc1 = ops.StdConv(C_p, C, 1, 1, 0)
        
        #generate dag
        if reduction:
            gene = genotype.reduce
            self.concat = genotype.reduce_concat
        else:
            gene = genotype.normal
            self.concat = genotype.normal_concat
        
        self.dag = gt.to_dag(C, gene, reduction)
        
    def forward(self, s0, s1):
        s0 = self.preproc0(s0)
        s1 = self.preproc1(s1)
        
        states = [s0, s1]
        for edges in self.dag:
            s_cur = sum(op(states[op.s_idx]) for op in edges)
            states.append(s_cur)
            
        s_out = torch.cat([states[i] for i in self.concat], dim=1)
        
        return s_out